In [1]:
import torch
import os
from pathlib import Path
import warnings
from tqdm import tqdm
from IPython.display import clear_output

from razdel import sentenize
import fitz
import re

import pandas as pd
import seaborn as sns

warnings.filterwarnings('ignore')

In [2]:
class SentsDataset(torch.utils.data.Dataset):
    
    @staticmethod
    def read_df(path, data_dir):
        df = pd.read_csv(os.path.join(f"{data_dir}/sents", path))
        source_name = path.split(".")[0]
        final_df = pd.DataFrame({"source_name": len(df)*[source_name],
                                "text": df.text})
        return final_df
    
    def __init__(self, data_dir="data"):
        self.data_dir = data_dir
        assert Path(f"{self.data_dir}/sents").is_dir() == True
        
        dataframes = []
        
        for path in os.listdir(f"{self.data_dir}/sents"):
            if os.path.isfile(os.path.join(f"{self.data_dir}/sents", path)) and path != "labeled.csv":
                dataframes.append(self.read_df(path, data_dir))
        
        self.df = pd.concat(dataframes)
        
    def sample(self, n=1):
        if n == 1:
            return self.df.sample()
        return self.df.sample(n)
    
    def __getitem__():
        pass
    
    def __len__():
        pass
        
    def append():
        pass
    
    def __contains__():
        pass

In [26]:
class LabeledDataset(SentsDataset):
    
    LABELS = {
        'y' : 1, # scientific
        'n' : 0  # non scientific
    }
    
    def __init__(self, data_dir="data"):
        self.data_dir = data_dir
        assert Path(f"{self.data_dir}/sents").is_dir() == True
        
        if not os.path.isfile(os.path.join(f"{self.data_dir}", "sents", "labeled.csv")):
            self.df = pd.DataFrame({'text': [], "source_name": [], 'label': []})
        else:
            self.df = pd.read_csv(os.path.join(f"{self.data_dir}", "sents", "labeled.csv"))
        
    def checkpoint(self):
        self.df.to_csv(os.path.join(f"{self.data_dir}", "sents", "labeled.csv"), index=False)
    
    def label_df(self, df):
        inp = "s"
        while inp != 'exit':
            record = df.sample()
            
            if record.text.item() in self.df.text:
                continue

            clear_output(wait=True)
            print(f'Source  : {record.source_name.item()}')
            print(f'Text    : {record.text.item()}')
            
            inp = input().lower()
            while inp not in ["y", "n", 's', 'exit']:
                print("invalid command")
                print("valid commands: Y, N, S, EXIT")
                inp = input().lower()
            
            if inp in ['y', 'n']:
                new_record = {'text': record.text.item(),
                              'source_name': record.source_name.item(),
                              'label': self.LABELS[inp]}
                self.df = self.df.append(new_record, ignore_index=True)
                self.checkpoint()
            
            if inp == "exit":
                self.checkpoint()    

In [22]:
sents = SentsDataset()

In [27]:
labeler = LabeledDataset()
labeler.label_df(sents)

Source  : habr
Text    : Порядок элементов в буфере обмена при этом может измениться.


KeyboardInterrupt: Interrupted by user

In [28]:
labeler.df

,text,source_name,label
0,Это создаст следующую структуру каталогов.,habr,0.0
1,Тогда в Греции все было хорошо.,habr,0.0
2,Как пользоваться git вообще и Github в частнос...,habr,0.0
3,Большая часть поста - про Волгоград.,habr,0.0
4,"Он нам нужен, чтобы настроить поток, но нас ни...",habr,0.0
...,...,...,...
171,Например: прерывание инициализаторов классов м...,spbu,1.0
172,"Полезный опыт, как никак.",habr,0.0
173,Я решил установить его на жёсткий диск.,habr,0.0
174,После этого его подход на практике продемонстр...,habr,1.0
